# Exploring the data

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt

from helpers import *

import warnings
warnings.filterwarnings("ignore")

#### Load the dataframe containing videos from 2019 and their features

This is a big file (860 MB) so we have stored it on Google Drive. Download it from the link below and storie it as `generated/2019/2019_videos_Typo_Emojis_NegWords_Sentiment_title_desc.parquet`. It was generated by the notebook `data_processing.ipynb`.

https://drive.google.com/file/d/1RmVSw2MBq0Ps0dwcTQjqZsDAuivXbUaZ/view?usp=share_link

In [ ]:
filepath = 'generated/2019/2019_videos_Typo_Emojis_NegWords_Sentiment_title_desc.parquet'
videos = pd.read_parquet(filepath, engine='fastparquet')

We check that each video has a unique identifier (column `display_id`).

In [ ]:
videos.set_index(videos['display_id']).index.is_unique

There are videos with NaN values in some field but we do not remove them as it is not needed for our analysis.

In [ ]:
nb_nan = len(videos) - len(videos.dropna())

print(f'There are {nb_nan} videos with a NaN field ({nb_nan / len(videos) :.1%} of the total).')

What fraction of the dataset do videos from 2019 respresent?

In [ ]:
nb_videos_2019 = len(videos)
nb_videos_total = 73e6

print(f'Fractions of videos that are from 2019: {(nb_videos_2019 / nb_videos_total):.1%}')

What columns do we have in our dataframe?

Fields that were already present in the original dataset from Zenodo:

- `categories`
- `channel_id`
- `crawl_date`
- `dislike_count`
- `display_id`
- `duration`
- `like_count`
- `upload_date`
- `view_count`

Features we have created:

- Number of exclamation marks in the title (or the description) of the video: `count_excl_marks_title`, `count_excl_marks_description`.

- Number of words in all caps in the title (or the description) of the video: `count_upper_words_title`, `count_upper_words_description`.

- Number of negative emojis in the title (or the description) of the video: `count_negative_emojis_title`, `count_negative_emojis_description`.

- Number of (negative) words in the title (or the description) of the video: `count_words_title`, `count_negative_words_title`, `count_words_description`, `count_negative_words_description`.

- Intensity score in the title (or the description) of the sentiment 'negative', 'neutral' and 'positive' according to VADER (plus compund): `sia_negative_title`, `sia_neutral_title`, `sia_positive_title`, `sia_compound_title`, `sia_negative_description`, `sia_neutral_description`, `sia_positive_description`, `sia_compound_description`.

In [ ]:
videos.info()

Let us look at some basic statistics of the data.

In [ ]:
videos.describe()

### Look at the distribution of the features

We look at ratios since what matters for the negativity is the fraction of negative words (etc.) and not the absolute count.

In [ ]:
# Prepare the field types (this will help us plot the data)

type_text_desc = ['description'] * len(videos)
type_text_title = ['title'] * len(videos)
type_text = np.concatenate((type_text_desc, type_text_title), None)

#### Typography of the text

In [ ]:
# Compute the ratios

ratio_excl_title = videos['count_excl_marks_title'] / videos['count_words_title']
ratio_excl_description = videos['count_excl_marks_description'] / videos['count_words_description']
ratio_excl = np.concatenate((ratio_excl_description, ratio_excl_title), None)
# Note: The ratio nb_exclamation_marks / nb_words is maybe not the best choice, we might change it later on.

ratio_upper_title = videos['count_upper_words_title']/videos['count_words_title']
ratio_upper_description = videos['count_upper_words_description']/videos['count_words_description']
ratio_upper_words = np.concatenate((ratio_upper_description, ratio_upper_title), None)


# Plot

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16,6))

ratio_typo = pd.DataFrame({
    'Field': type_text,
    'Fraction of exclamation marks (out of words)': ratio_excl,
    'Fraction of upper words': ratio_upper_words,
})

sns.histplot(ratio_typo, x='Fraction of exclamation marks (out of words)', bins=50, hue="Field", element="step", ax=axes[1]).set(yscale ='log')
sns.histplot(ratio_typo, x='Fraction of upper words', bins=50, hue="Field", element="step", ax=axes[0]).set(yscale ='log')

fig.suptitle('Typographic features')
plt.tight_layout()
plt.show()

#### Emojis

In [ ]:
# Compute the ratios

ratio_emojis_title = videos['count_negative_emojis_title'] / videos['count_words_title']
ratio_emojis_description = videos['count_negative_emojis_description'] / videos['count_words_description']
ratio_emojis = np.concatenate((ratio_emojis_description, ratio_emojis_title), None)


# Plot

ratio_emojis = pd.DataFrame({
    'Field': type_text,
    'Fraction of negative emojis (out of words)': ratio_emojis
})

fig = plt.subplot()
sns.histplot(ratio_emojis, x='Fraction of negative emojis (out of words)', bins=50, hue="Field", element="step").set(yscale ='log')

fig.set_title('Negative emojis')
plt.tight_layout()
plt.show()

#### Negative words

In [ ]:
# Compute the ratios

ratio_neg_words_title = videos['count_negative_words_title']/videos['count_words_title']
ratio_neg_words_description = videos['count_negative_words_description']/videos['count_words_description']


# Plot

ratio_neg_words = pd.DataFrame({
    'Field': type_text,
    'Fraction of negative words': np.concatenate((ratio_neg_words_description, ratio_neg_words_title), None)
})

fig = plt.subplot()
sns.histplot(ratio_neg_words, x='Fraction of negative words', bins=50, hue="Field", element="step").set(yscale ='log')

fig.set_title('Negative words')
plt.tight_layout()
plt.show()

#### Sentiment intensity

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(16,12))

sentiment = pd.DataFrame({
    'Field': type_text,
    'Negative sentiment': np.concatenate((videos['sia_negative_description'], videos['sia_negative_title']), None),
    'Positive sentiment': np.concatenate((videos['sia_positive_description'], videos['sia_positive_title']), None),
    'Neutral sentiment': np.concatenate((videos['sia_neutral_description'], videos['sia_neutral_title']), None),
    'Compound': np.concatenate((videos['sia_compound_description'], videos['sia_compound_title']), None)

})

sns.histplot(sentiment, x='Negative sentiment', bins=50, hue="Field", element="step", ax=axes[0, 0]).set(yscale ='log')
sns.histplot(sentiment, x='Positive sentiment', bins=50, hue="Field", element="step", ax=axes[0, 1]).set(yscale ='log')
sns.histplot(sentiment, x='Neutral sentiment', bins=50, hue="Field", element="step", ax=axes[1, 0]).set(yscale ='log')
sns.histplot(sentiment, x='Compound', bins=50, hue="Field", element="step", ax=axes[1, 1]).set(yscale ='log')

fig.suptitle('Sentiment intensity (Vader)')
plt.tight_layout()
plt.show()

#### Combine timeseries and videos to get ratios 

For each video, we want to get the ratio of like by subscribers of the channel that uploaded the video. As well as the ratio of dislike by subscribers, and views by total views of the channel. To do this, we need to merge the timeseries dataframe and the videos features dataframe on the date, grouping them by channel id.

Since we want to get the most accurate number of subscribers at the time of upload of the given video, we perform a nearest time merge, where we only merge on the entry of timeseries that has the closest datetime to the upload date, given a tolerance of one week (as the timeseries analysis was conducted per week). 

First step is to load the timeseries and filter out the years that are not used in our study (here we only keep 2019 for the moment)

In [ ]:
df_timeseries = pd.read_csv("data/df_timeseries_en.tsv", sep="\t")
df_timeseries_2019 = df_timeseries[pd.to_datetime(df_timeseries["datetime"]).dt.year == 2019]

In [ ]:
df_timeseries_2019.head(5)

Then, we rename the columns accordingly for the merge asof

In [ ]:
df_timeseries_2019["datetime"] = pd.to_datetime(df_timeseries_2019['datetime'])
df_timeseries_2019.rename({"datetime": "nearest_time"}, axis=1, inplace=True)
videos.rename({"upload_date": "nearest_time"}, axis=1, inplace=True)
df_timeseries_2019.rename({"channel": "channel_id"}, axis=1, inplace=True)

In [ ]:

videos_on_date_by_channel = pd.merge_asof(videos.sort_values("nearest_time"),df_timeseries_2019.sort_values("nearest_time"),on="nearest_time",by="channel_id",allow_exact_matches=True,direction="nearest",tolerance=pd.Timedelta(1,unit="W"))

Since some videos were uploaded by channels that are not in the timeseries, we end up with NaN values. We decided to remove the rows that have these NaN values.

In [ ]:
# approx 1M missing values (channels that are in metadata but not in timeseries)
videos_on_date_by_channel.dropna(axis=0,inplace=True)

We create new columns for the ratios

In [ ]:
def create_ratio_column(df, new_col,col1,col2):
        df[new_col] = df[col1]/df[col2]

create_ratio_column(videos_on_date_by_channel,"like_count_by_subs","like_count","subs")
create_ratio_column(videos_on_date_by_channel,"dislike_count_by_subs","dislike_count","subs")
create_ratio_column(videos_on_date_by_channel,"view_count_by_channel_views","view_count","views")
create_ratio_column(videos_on_date_by_channel,"view_count_by_channel_subs","view_count","subs")

Let's now plot the distribution of the number of views between all the videos and the distribution of the number of subscribers between all the channels. In fact, knowing this information will helps us decide what kind of normalization should be use in the rest of the project.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16,6))

videos_on_date_by_channel = videos_on_date_by_channel[videos_on_date_by_channel["views"] > 0]
videos_on_date_by_channel = videos_on_date_by_channel[videos_on_date_by_channel["subs"] > 0]

sns.histplot(videos_on_date_by_channel, x="views", bins=50, element="step", ax=axes[0])
sns.histplot(videos_on_date_by_channel, x="subs", bins=50, element="step", ax=axes[1])

axes[0].set(yscale ='log')
axes[0].set_title('Distribution of the number of views between all the videos')

axes[1].set(yscale ='log')
axes[1].set_title('Distribution of the number of subscribers between all the channels')

plt.tight_layout()
plt.show()

In [ ]:
videos_on_date_by_channel_activity =  videos_on_date_by_channel.loc[videos_on_date_by_channel['activity'] == 1]

In [ ]:
videos_on_date_by_channel_activity.columns

In [ ]:
sns.histplot(videos_on_date_by_channel, x="subs", bins=50, element="step", log_scale=(False,True))


In [ ]:
videos_on_date_by_channel['neg_title_bool'] = videos_on_date_by_channel['sia_negative_title'].apply(lambda x: 'negative' if x > 0.5 else 'positive')

videos_on_date_by_channel['neg_desc_bool'] = videos_on_date_by_channel['sia_negative_description'].apply(lambda x: 'negative' if x > 0.5 else 'positive')

In [ ]:
videos_on_date_by_channel['neg_title_bool']

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16,12))



sns.histplot(videos_on_date_by_channel, x='view_count_by_channel_subs', bins=5, hue="neg_title_bool", element="step", ax=axes[0]).set(yscale ='log')
sns.histplot(videos_on_date_by_channel, x='view_count_by_channel_subs', bins=5, hue="neg_desc_bool", element="step", ax=axes[1]).set(yscale ='log')

fig.suptitle('view count per channel depending negativity')
plt.tight_layout()
plt.show()

In [ ]:
df_grouped = videos_on_date_by_channel.groupby('channel_id', as_index=False)[['sia_negative_title', 'sia_neutral_title', 'sia_positive_title',
       'sia_compound_title', 'sia_negative_description',
       'sia_neutral_description', 'sia_positive_description',
       'sia_compound_description', 'views', 'delta_views', 'subs',
       'delta_subs','like_count_by_subs', 'dislike_count_by_subs',
       'view_count_by_channel_views', 'view_count_by_channel_subs']].mean()

In [ ]:
df_grouped

In [ ]:
df_grouped.loc[df_grouped['subs'] < 0]



In [ ]:
def classify_channel(df):
    new_column = []
    for idx, row in df.iterrows():
       
        
        
        if row['subs']<1000:
            new_column.append('small')
            
        elif row['subs']>=1000 and row['subs']<10000:
            new_column.append('rather small')
            
        elif row['subs']>=10000 and row['subs']<100000:
            new_column.append('medium')
            
        elif row['subs']>=100000 and row['subs']<1000000:
            new_column.append('rather big')
            
        elif row['subs']>=1000000:
            new_column.append('big')
            
            
    df2 = df.assign(SizeChannel=new_column)
    return df2



In [ ]:
new_df_grouped = classify_channel(df_grouped)

In [ ]:
new_df_grouped.loc[new_df_grouped['sia_negative_title'] > 0.5]

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16,12))



sns.histplot(new_df_grouped, x='view_count_by_channel_subs', bins=5, hue="SizeChannel", element="step", ax=axes[0]).set(yscale ='log')
sns.histplot(new_df_grouped, x='view_count_by_channel_subs', bins=5, hue="SizeChannel", element="step", ax=axes[1]).set(yscale ='log')

fig.suptitle('view count per channel depending size channel')
plt.tight_layout()
plt.show()

In [ ]:
new_df_grouped['neg_title_bool'] = new_df_grouped['sia_negative_title'].apply(lambda x: 'negative' if x > 0.5 else 'positive')

new_df_grouped['neg_desc_bool'] = new_df_grouped['sia_negative_description'].apply(lambda x: 'negative' if x > 0.5 else 'positive')



In [ ]:
df_small = new_df_grouped.loc[new_df_grouped['SizeChannel'] == 'small']
df_rather_small = new_df_grouped.loc[new_df_grouped['SizeChannel'] == 'rather small']
df_medium = new_df_grouped.loc[new_df_grouped['SizeChannel'] == 'medium']
df_rather_big = new_df_grouped.loc[new_df_grouped['SizeChannel'] == 'rather big']
df_big = new_df_grouped.loc[new_df_grouped['SizeChannel'] == 'big']

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(16,12))



sns.histplot(df_small, x='view_count_by_channel_subs', bins=1000, hue="neg_title_bool", element="step", ax=axes[0, 0]).set(yscale ='log')
sns.histplot(df_small, x='view_count_by_channel_subs', bins=1000, hue="neg_desc_bool", element="step", ax=axes[0,1]).set(yscale ='log')

sns.histplot(df_rather_small, x='view_count_by_channel_subs', bins=1000, hue="neg_title_bool", element="step", ax=axes[1, 0]).set(yscale ='log')
sns.histplot(df_rather_small, x='view_count_by_channel_subs', bins=1000, hue="neg_desc_bool", element="step", ax=axes[1,1]).set(yscale ='log')

sns.histplot(df_medium, x='view_count_by_channel_subs', bins=1000, hue="neg_title_bool", element="step", ax=axes[2, 0]).set(yscale ='log')
sns.histplot(df_medium, x='view_count_by_channel_subs', bins=1000, hue="neg_desc_bool", element="step", ax=axes[2,1]).set(yscale ='log')

sns.histplot(df_rather_big, x='view_count_by_channel_subs', bins=1000, hue="neg_title_bool", element="step", ax=axes[3, 0]).set(yscale ='log')
sns.histplot(df_rather_big, x='view_count_by_channel_views', bins=1000, hue="neg_desc_bool", element="step", ax=axes[3,1]).set(yscale ='log')

sns.histplot(df_big, x='view_count_by_channel_subs', bins=1000, hue="neg_title_bool", element="step", ax=axes[4, 0]).set(yscale ='log')
sns.histplot(df_big, x='view_count_by_channel_subs', bins=1000, hue="neg_desc_bool", element="step", ax=axes[4,1]).set(yscale ='log')


axes[0, 0].set_title('view_count_by_channel_subs depending negativity in title for small channels')
axes[0, 1].set_title('view_count_by_channel_subs depending negativity in description for small channels')

axes[1, 0].set_title('view_count_by_channel_subs depending negativity in title for rather small channels')
axes[1, 1].set_title('view_count_by_channel_subs depending negativity in description for rather small channels')

axes[2, 0].set_title('view_count_by_channel_subs depending negativity in title for medium channels')
axes[2, 1].set_title('view_count_by_channel_subs depending negativity in description for medium channels')


axes[3, 0].set_title('view_count_by_channel_subs depending negativity in title for rather big channels')
axes[3, 1].set_title('view_count_by_channel_subs depending negativity in description for rather big channels')

axes[4, 0].set_title('view_count_by_channel_subs depending negativity in title for big channels')
axes[4, 1].set_title('view_count_by_channel_subs depending negativity in description for big channels')
fig.suptitle('view count per channel depending negativity and size of channel')
plt.tight_layout()
plt.show()

In [ ]:



df_small_neg_title = df_small.loc[df_small['neg_title_bool'] == 'negative']
df_small_neg_description = df_small.loc[df_small['neg_desc_bool'] == 'negative']

df_rather_small_neg_title = df_rather_small.loc[df_rather_small['neg_title_bool'] == 'negative']
df_rather_small_neg_description = df_rather_small.loc[df_rather_small['neg_desc_bool'] == 'negative']

df_medium_neg_title = df_medium.loc[df_medium['neg_title_bool'] == 'negative']
df_medium_neg_description = df_medium.loc[df_medium['neg_desc_bool'] == 'negative']

df_rather_big_neg_title = df_rather_big.loc[df_rather_big['neg_title_bool'] == 'negative']
df_rather_big_neg_description = df_rather_big.loc[df_rather_big['neg_desc_bool'] == 'negative']

df_big_neg_title = df_big.loc[df_big['neg_title_bool'] == 'negative']
df_big_neg_description = df_big.loc[df_big['neg_desc_bool'] == 'negative']



In [ ]:





fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(16,12))



sns.histplot(df_small_neg_title, x='view_count_by_channel_subs', bins=50, element="step", ax=axes[0, 0]).set(yscale ='log')
sns.histplot(df_small_neg_description, x='view_count_by_channel_subs', bins=50, element="step", ax=axes[0,1]).set(yscale ='log')

sns.histplot(df_rather_small_neg_title, x='view_count_by_channel_subs', bins=50, element="step", ax=axes[1, 0]).set(yscale ='log')
sns.histplot(df_rather_small_neg_description, x='view_count_by_channel_subs', bins=50, element="step", ax=axes[1,1]).set(yscale ='log')

sns.histplot(df_medium_neg_title, x='view_count_by_channel_subs', bins=50, element="step", ax=axes[2, 0]).set(yscale ='log')
sns.histplot(df_medium_neg_description, x='view_count_by_channel_subs', bins=50, element="step", ax=axes[2,1]).set(yscale ='log')

sns.histplot(df_rather_big_neg_title, x='view_count_by_channel_subs', bins=50, element="step", ax=axes[3, 0]).set(yscale ='log')
sns.histplot(df_rather_big_neg_description, x='view_count_by_channel_subs', bins=50, element="step", ax=axes[3,1]).set(yscale ='log')

sns.histplot(df_big_neg_title, x='view_count_by_channel_subs', bins=50, element="step", ax=axes[4, 0]).set(yscale ='log')
sns.histplot(df_big_neg_description, x='view_count_by_channel_subs', bins=50, element="step", ax=axes[4,1]).set(yscale ='log')


axes[0, 0].set_title('view_count_by_channel_subs for negative title and small channels')
axes[0, 1].set_title('view_count_by_channel_subs for negative description and small channels')

axes[1, 0].set_title('view_count_by_channel_subs for negative title and rather small channels')
axes[1, 1].set_title('view_count_by_channel_subs for negative description and rather small channels')

axes[2, 0].set_title('view_count_by_channel_subs for negative title and medium channels')
axes[2, 1].set_title('view_count_by_channel_subs for negative description and medium channels')


axes[3, 0].set_title('view_count_by_channel_subs for negative title and rather big channels')
axes[3, 1].set_title('view_count_by_channel_subs for negative description and rather big channels')

axes[4, 0].set_title('view_count_by_channel_subs for negative title and big channels')
axes[4, 1].set_title('view_count_by_channel_subs for negative description and big channels')
fig.suptitle('view count per channel depending negativity and size of channel')
plt.tight_layout()
plt.show()

In [ ]:
videos_on_date_by_channel_activity['neg_title_bool'] = videos_on_date_by_channel_activity['sia_negative_title'].apply(lambda x: 'negative' if x > 0.5 else 'positive')

videos_on_date_by_channel_activity['neg_desc_bool'] = videos_on_date_by_channel_activity['sia_negative_description'].apply(lambda x: 'negative' if x > 0.5 else 'positive')

In [ ]:
new_df_activity = classify_channel(videos_on_date_by_channel_activity)


In [ ]:
df_small_activity = new_df_activity.loc[new_df_activity['SizeChannel'] == 'small']
df_rather_small_activity = new_df_activity.loc[new_df_activity['SizeChannel'] == 'rather small']
df_medium_activity = new_df_activity.loc[new_df_activity['SizeChannel'] == 'medium']
df_rather_big_ativity = new_df_activity.loc[new_df_activity['SizeChannel'] == 'rather big']
df_big_activity = new_df_activity.loc[new_df_activity['SizeChannel'] == 'big']

In [ ]:
df_small_neg_title_activity = df_small_activity.loc[df_small_activity['neg_title_bool'] == 'negative']
df_small_neg_description_activity = df_small_activity.loc[df_small_activity['neg_desc_bool'] == 'negative']

df_rather_small_neg_title_activity = df_rather_small_activity.loc[df_rather_small_activity['neg_title_bool'] == 'negative']
df_rather_small_neg_description_activity = df_rather_small_activity.loc[df_rather_small_activity['neg_desc_bool'] == 'negative']

df_medium_neg_title_activity = df_medium_activity.loc[df_medium_activity['neg_title_bool'] == 'negative']
df_medium_neg_description_activity = df_medium_activity.loc[df_medium_activity['neg_desc_bool'] == 'negative']

df_rather_big_neg_title_activity = df_rather_big_ativity.loc[df_rather_big_ativity['neg_title_bool'] == 'negative']
df_rather_big_neg_description_activity = df_rather_big_ativity.loc[df_rather_big_ativity['neg_desc_bool'] == 'negative']

df_big_neg_title_activity = df_big_activity.loc[df_big_activity['neg_title_bool'] == 'negative']
df_big_neg_description_activity = df_big_activity.loc[df_big_activity['neg_desc_bool'] == 'negative']



In [ ]:
df_small_neg_title_activity = df_small_activity.loc[df_small_activity['neg_title_bool'] == 'negative']
df_small_neg_description_activity = df_small_activity.loc[df_small_activity['neg_desc_bool'] == 'positive']

df_rather_small_neg_title_activity = df_rather_small_activity.loc[df_rather_small_activity['neg_title_bool'] == 'positive']
df_rather_small_neg_description_activity = df_rather_small_activity.loc[df_rather_small_activity['neg_desc_bool'] == 'positive']

df_medium_neg_title_activity = df_medium_activity.loc[df_medium_activity['neg_title_bool'] == 'positive']
df_medium_neg_description_activity = df_medium_activity.loc[df_medium_activity['neg_desc_bool'] == 'positive']

df_rather_big_neg_title_activity = df_rather_big_ativity.loc[df_rather_big_ativity['neg_title_bool'] == 'positive']
df_rather_big_neg_description_activity = df_rather_big_ativity.loc[df_rather_big_ativity['neg_desc_bool'] == 'positive']

df_big_neg_title_activity = df_big_activity.loc[df_big_activity['neg_title_bool'] == 'positive']
df_big_neg_description_activity = df_big_activity.loc[df_big_activity['neg_desc_bool'] == 'positive']

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(16,12))



sns.histplot(df_small_activity, x='delta_subs', bins=1000, hue="neg_title_bool", element="step", ax=axes[0, 0]).set(yscale ='log')
sns.histplot(df_small_activity, x='delta_subs', bins=1000, hue="neg_desc_bool", element="step", ax=axes[0,1]).set(yscale ='log')

sns.histplot(df_rather_small_activity, x='delta_subs', bins=1000, hue="neg_title_bool", element="step", ax=axes[1, 0]).set(yscale ='log')
sns.histplot(df_rather_small_activity, x='delta_subs', bins=1000, hue="neg_desc_bool", element="step", ax=axes[1,1]).set(yscale ='log')

sns.histplot(df_medium_activity, x='delta_subs', bins=1000, hue="neg_title_bool", element="step", ax=axes[2, 0]).set(yscale ='log')
sns.histplot(df_medium_activity, x='delta_subs', bins=1000, hue="neg_desc_bool", element="step", ax=axes[2,1]).set(yscale ='log')

sns.histplot(df_rather_big_ativity, x='delta_subs', bins=1000, hue="neg_title_bool", element="step", ax=axes[3, 0]).set(yscale ='log')
sns.histplot(df_rather_big_ativity, x='delta_subs', bins=1000, hue="neg_desc_bool", element="step", ax=axes[3,1]).set(yscale ='log')

sns.histplot(df_big_activity, x='delta_subs', bins=1000, hue="neg_title_bool", element="step", ax=axes[4, 0]).set(yscale ='log')
sns.histplot(df_big_activity, x='delta_subs', bins=1000, hue="neg_desc_bool", element="step", ax=axes[4,1]).set(yscale ='log')


axes[0, 0].set_title('view_count_by_channel_subs depending negativity in title for small channels')
axes[0, 1].set_title('view_count_by_channel_subs depending negativity in description for small channels')

axes[1, 0].set_title('view_count_by_channel_subs depending negativity in title for rather small channels')
axes[1, 1].set_title('view_count_by_channel_subs depending negativity in description for rather small channels')

axes[2, 0].set_title('view_count_by_channel_subs depending negativity in title for medium channels')
axes[2, 1].set_title('view_count_by_channel_subs depending negativity in description for medium channels')


axes[3, 0].set_title('view_count_by_channel_subs depending negativity in title for rather big channels')
axes[3, 1].set_title('view_count_by_channel_subs depending negativity in description for rather big channels')

axes[4, 0].set_title('view_count_by_channel_subs depending negativity in title for big channels')
axes[4, 1].set_title('view_count_by_channel_subs depending negativity in description for big channels')
fig.suptitle('view count per channel depending negativity and size of channel')
plt.tight_layout()
plt.show()

In [ ]:
N = int(1e7)
B = 3
fig, axes = plt.subplots(1, 3, figsize=(10,3))
short = videos_drop_likena[:N]
axes[0].hexbin(short['sia_negative_title'], short['like_count'], bins=B)
axes[0].set_title('sia_negative_title')
axes[1].hexbin(short['sia_positive_title'], short['like_count'], bins=B)
axes[1].set_title('sia_positive_title')
axes[2].hexbin(short['sia_neutral_title'], short['like_count'], bins=B)
axes[2].set_title('sia_neutral_title')
plt.tight_layout()
plt.show()

In [ ]:
import statsmodels.formula.api as smf


def print_regression(data, formula):
    model = smf.ols(formula=formula, data=data)
    np.random.seed(2)
    results = model.fit()
    print(results.summary())


N = int(1e7)
videos_drop_likena = videos[videos['like_count'].isna() == False]
short = videos_drop_likena[:N]
short = videos_drop_likena


short['like_count'] = short['like_count'].astype(float)
short['dislike_count'] = short['dislike_count'].astype(float)
short['view_count'] = short['view_count'].astype(float)
short['log_view_count'] = np.log(short['view_count'] + 2)

In [ ]:
f = 'log_view_count ~ '
f += 'sia_negative_description ' 
f += '+ sia_positive_description '
f += '+ sia_neutral_description '


print_regression(data=short, formula=f)

# interesting results: (log)_like_count, dislike_count, log_view_count

In [ ]:
categories = set(short['categories'].values)

In [ ]:
short_news_politics = short[short['categories'] == 'News & Politics']

print_regression(data=short_news_politics, formula=f)


In [ ]:
short_animals = short[short['categories'] == 'Pets & Animals']

print_regression(data=short_animals, formula=f)

1. Make 'histogram' of words in title/desc for videos that are very negative and have lots of success (maybe do it for each category)

2. 
